In [0]:
%python
dbutils.widgets.removeAll()

In [0]:
create widget text storageLocation default "abfss://lhclproject@adlsmartdata2446.dfs.core.windows.net/";

### Creacion de catalog

In [0]:
CREATE EXTERNAL LOCATION IF NOT EXISTS `exlt-lhclproject`
URL '${storageLocation}'
WITH (STORAGE CREDENTIAL `credential`)
COMMENT 'Ubicación externa para el catalogo del proyecto';

In [0]:
CREATE CATALOG IF NOT EXISTS final_project_flights
MANAGED LOCATION '${storageLocation}';

### Creacion de Schemas

In [0]:
USE CATALOG final_project_flights;

CREATE SCHEMA IF NOT EXISTS raw;
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS golden;

### Creacion tablas Bronze

In [0]:
USE CATALOG final_project_flights;
USE SCHEMA bronze;

CREATE OR REPLACE TABLE airlines (
  IATA_CODE STRING,
  AIRLINE STRING
)
USING DELTA
LOCATION '${storageLocation}/bronze/airlines';

CREATE OR REPLACE TABLE airports (
  IATA_CODE STRING,
  AIRPORT STRING,
  CITY STRING,
  STATE STRING,
  COUNTRY STRING,
  LATITUDE STRING,
  LONGITUDE STRING
)
USING DELTA
LOCATION '${storageLocation}/bronze/airports';

CREATE OR REPLACE TABLE flights (
    YEAR INT,
    MONTH INT,
    DAY INT,
    DAY_OF_WEEK INT,
    AIRLINE STRING,
    FLIGHT_NUMBER INT,
    TAIL_NUMBER STRING,
    ORIGIN_AIRPORT STRING,
    DESTINATION_AIRPORT STRING,
    SCHEDULED_DEPARTURE INT,
    DEPARTURE_TIME INT,
    DEPARTURE_DELAY INT,
    TAXI_OUT INT,
    WHEELS_OFF INT,
    SCHEDULED_TIME INT,
    ELAPSED_TIME INT,
    AIR_TIME INT,
    DISTANCE INT,
    WHEELS_ON INT,
    TAXI_IN INT,
    SCHEDULED_ARRIVAL INT,
    ARRIVAL_TIME INT,
    ARRIVAL_DELAY INT,
    DIVERTED INT,
    CANCELLED INT,
    CANCELLATION_REASON STRING,
    AIR_SYSTEM_DELAY INT,
    SECURITY_DELAY INT,
    AIRLINE_DELAY INT,
    LATE_AIRCRAFT_DELAY INT,
    WEATHER_DELAY INT
)
USING DELTA
LOCATION '${storageLocation}/bronze/flights';

### Creacion tablas Silver

In [0]:
USE CATALOG final_project_flights;
USE SCHEMA silver;

CREATE OR REPLACE TABLE airports_silver (
  IATA_CODE STRING,
  AIRPORT STRING,
  CITY STRING,
  STATE STRING,
  COUNTRY STRING,
  LATITUDE DOUBLE,
  LONGITUDE DOUBLE
)
USING DELTA
LOCATION '${storageLocation}/silver/airports';

CREATE OR REPLACE TABLE flights_silver (
  DAY_OF_WEEK INT,
  AIRLINE STRING,
  FLIGHT_NUMBER INT,
  TAIL_NUMBER STRING,
  ORIGIN_AIRPORT STRING,
  DESTINATION_AIRPORT STRING,
  DEPARTURE_DELAY INT,
  TAXI_OUT INT,
  SCHEDULED_TIME INT,
  ELAPSED_TIME INT,
  AIR_TIME INT,
  DISTANCE INT,
  TAXI_IN INT,
  ARRIVAL_DELAY INT,
  DIVERTED INT,
  CANCELLED INT,
  CANCELLATION_REASON STRING,
  AIR_SYSTEM_DELAY INT,
  SECURITY_DELAY INT,
  AIRLINE_DELAY INT,
  LATE_AIRCRAFT_DELAY INT,
  WEATHER_DELAY INT,
  SCHEDULED_DEPARTURE_TS TIMESTAMP,
  SCHEDULED_ARRIVAL_TS TIMESTAMP,
  DEPARTURE_TIME_TS TIMESTAMP,
  ARRIVAL_TIME_TS TIMESTAMP,
  AIRLINE_NAME STRING,
  ORIGIN_CITY STRING,
  ORIGIN_STATE STRING,
  ORIGIN_LAT DOUBLE,
  ORIGIN_LON DOUBLE,
  DESTINATION_CITY STRING,
  DESTINATION_STATE STRING,
  DESTINATION_LAT DOUBLE,
  DESTINATION_LON DOUBLE
)
USING DELTA
LOCATION '${storageLocation}/silver/flights';

### Creacion tablas Golden

In [0]:
USE CATALOG final_project_flights;
USE SCHEMA golden;

CREATE OR REPLACE TABLE TM_TIEMPO (
  DATE_KEY INT,
  FULL_DATE DATE,
  DAY_OF_WEEK INT,
  DAY_NAME STRING,
  DAY_OF_MONTH INT,
  WEEK_OF_YEAR INT,
  MONTH INT,
  QUARTER INT,
  YEAR INT
)
USING DELTA
LOCATION '${storageLocation}/golden/tm_tiempo';

CREATE OR REPLACE TABLE TM_AIRPORTS (
  AIRPORT_KEY STRING,
  AIRPORT STRING,
  CITY STRING,
  STATE STRING,
  GEO_LAT DOUBLE,
  GEO_LONG DOUBLE
)
USING DELTA
LOCATION '${storageLocation}/golden/tm_airports';

CREATE OR REPLACE TABLE RPT_RESUMEN_VUELOS_DIARIO (
  DATE_KEY INT,
  SCHEDULED_DATE DATE,
  AIRLINE STRING,
  MAYOR_CAUSA_RETRASO STRING,
  TOTAL_FLIGHTS BIGINT,
  TOTAL_DISTANCE_MILES DOUBLE,
  TOTAL_CANCELLED BIGINT,
  TOTAL_DIVERTED BIGINT,
  AVG_DEPARTURE_DELAY_MIN DOUBLE,
  AVG_ARRIVAL_DELAY_MIN DOUBLE,
  FLIGHTS_ON_TIME_COUNT BIGINT,
  CANCELLATION_RATE DOUBLE,
  ON_TIME_PERFORMANCE_RATE DOUBLE
)
USING DELTA
LOCATION '${storageLocation}/golden/rpt_resumen_vuelos_diario';

CREATE OR REPLACE TABLE TR_PERFORMANCE_RUTA (
  YEAR_MONTH INT,
  AIRLINE STRING,
  ORIGIN_AIRPORT STRING,
  DESTINATION_AIRPORT STRING,
  TIPO_RUTA STRING,
  TOTAL_FLIGHTS_ROUTE BIGINT,
  AVG_AIR_TIME_MIN DOUBLE,
  AVG_DEPARTURE_DELAY_MIN DOUBLE,
  AVG_ARRIVAL_DELAY_MIN DOUBLE,
  AVG_DISTANCE_MILES DOUBLE
)
USING DELTA
LOCATION '${storageLocation}/golden/tr_performance_ruta';